# Assembler

In [1]:
def preprocess_assembly_code(assembly_code):
    assembly_code = assembly_code.splitlines()

    line_no=0
    line_alias={}

    #Line alias
    for line in assembly_code:
        s = line.strip().split(' ')
        if not s[0]:
            continue
        if s[0][-1]==':':
            line_alias[s[0][0:-1]]=line_no

        line_no+=1
    # print("line_alias:   ",line_alias, '\n')

    #Preprocessing (Removing alias)
    preprocessd_assembly_code = []
    for line in assembly_code:
        s = line.strip().split(' ')
        if not s[0]:
            continue
        if s[0][-1]==':':
            s=s[1:]
        # print(s)        

        inst = s[0]
        if inst == 'IF_P' or inst == 'ELSE_P':     #IF_P or ELSE_P
            out= [inst , line_alias[s[1]] ]
        else:
            out = s

        preprocessd_assembly_code.append(out)

    return preprocessd_assembly_code

# preprocess_assembly_code(assembly_code)

In [2]:
class SPCore():
    def __init__(self, THRD_ID):
        # self.R = [0 for i in range(16)]
        self.R={
                'R0'    :0,'R1'    :0,'R2'    :0,'R3'    :0,'R4'    :0,'R5'    :0,'R6'    :0,'R7'    :0,                         'R8'    :0,'R9'    :0,'R10'    :0,'R11'    :0,'R12'    :0,'R13'    :0,'R14'    :0,
                'R15'    :0
        }
        self.THRD_ID = THRD_ID
        self.P = True
        # self.P_stack = [True]

    def __str__(self):
        return (
            "SPCore:\t"+str(self.THRD_ID)+'\n'+
            "P\t"+str(self.P)+'\n'+
            # "P_stack\t"+str(self.P_stack)+'\n'+
            str(self.R)+'\n'
        )


In [6]:
class SMCores():
    def __init__(self,cores):
        self.cores = cores

        self.P_stack = [[True for _ in (self.cores)]]

    def __getitem__(self, idx):
        return self.cores[idx]
    def __len__(self):
        return len(self.cores)

    def info(self, debugVals):
        print('P_stack',self.P_stack)
        # print(debugVals)
        for r in list(debugVals['R']):
            print(r, end='  ')
            for core in self.cores:
                print('Core'+str(core.THRD_ID)+':',core.R[r], end='  ')
            print('')

    ##
    def pushStack(self): 
        new_mask=[]
        for i in range(len(self.cores)):
            p_ = self.P_stack[-1][i]
            if p_ == False:
                new_mask.append('X') #Consider 'X' state
            else:
                new_mask.append(self.cores[i].P)
        self.P_stack.append(new_mask)

    def compStack(self): 
        for i in range(len(self.cores)):
            p_ = self.P_stack[-1][i]
            if p_ == False:
                self.P_stack[-1][i] = True
            elif p_ == True:
                self.P_stack[-1][i] = False                


    def popStack(self):
        self.P_stack.pop()

    def validCores(self):
        res = []
        for i in range(len(self.cores)):
            p_ = self.P_stack[-1][i]
            if p_ == True:
                res.append(self.cores[i])   
        return res

    def all_p_false(self):
        for p_ in self.P_stack[-1]:
            if p_==True:
                return False
        return True

    def all_p_true(self):
        for p_ in self.P_stack[-1]:
            if p_==False:
                return False
        return True


In [7]:
# sm = SMCores([SPCore(0)])
# debugVals={"R":['R5','R6']}
# sm.info( debugVals)

In [8]:
class Scheduler():
    def __init__(self,f_path, cores, debugVals):
        self.assembly_code = open(f_path+"/prog.txt", "r").read()
        self.assembly_code = preprocess_assembly_code(self.assembly_code)
        print(self.assembly_code, '\n')

        self.DataMem = list(map(int,open(f_path+"/data.txt", "r").read().splitlines()))

        self.SM = SMCores(cores)

        self.PC = 0

        self.debugVals = debugVals

    def runProgram(self):        
        while self.PC <len(self.assembly_code):
            print('\nPC',self.PC)
            self.execute(self.assembly_code[self.PC])
            self.SM.info(self.debugVals)
            
            # print(self.SM[0])
        print('\n____Finished___')

    def execute(self, s):
        print(s)        
        # print('num of valid cores',len(self.SM.validCores()))

        opcode = s[0]
        #Normal Instructions - Runs only on valid (mask==1) cores
        if opcode == 'LOAD':
            x=s[1]
            y=s[2]
            # self.R[x]=DataMem[self.R[y]]
            for core in self.SM.validCores():
                core.R[x]=self.DataMem[core.R[y]] #TODO check mask
        elif opcode == 'LOADI':
            x=s[1]
            I=int(s[2])
            # self.R[x]=I
            for core in self.SM.validCores():
                core.R[x]=I
        elif opcode == 'LOADC':
            x=s[1]
            c=s[2]
            if c=='THRD_ID': #TODO:add rest
                # self.R[x]=self.THRD_ID
                for core in self.SM.validCores():
                    core.R[x]=core.THRD_ID
        elif opcode == 'STORE':
            x=s[1]
            y=s[2]
            for core in self.SM.validCores():
                self.DataMem[core.R[y]] = core.R[x]
        elif opcode == 'SETP':
            op = s[1]
            x = s[2] 
            y = s[3]
            if op=='EQ': 
                for core in self.SM.validCores():
                    core.P = (core.R[x]==core.R[y])
            elif op=='GT': #TODO: Add rest
                for core in self.SM.validCores():
                    core.P = (core.R[x]>core.R[y])


        #Branching Instructions
        elif opcode == 'IF_P':
            L = int(s[1])
            self.SM.pushStack()
            if self.SM.all_p_false():
                self.PC = L
                return


        elif opcode == 'ELSE_P':
            L = int(s[1])
            self.SM.compStack()
            if self.SM.all_p_true():
                self.PC = L
                return

        elif opcode == 'ENDIF':
            # popStack(self.cores)
            self.SM.popStack()

        else:
            raise Exception("Unknown Instruction")


        self.PC+=1
        return

In [9]:
# f_path = 'sample/1-simple-branch'
f_path = 'sample/2-nested-branch'

gpu = Scheduler(f_path, cores = [
    SPCore(0), SPCore(1), SPCore(2), SPCore(3)
],
debugVals={
    'R':['R5','R6','R7']
})

gpu.runProgram()

print(gpu.DataMem)

[['LOADC', 'R0', 'THRD_ID'], ['LOAD', 'R5', 'R0'], ['LOADI', 'R6', '1'], ['LOADI', 'R7', '3'], ['SETP', 'GT', 'R5', 'R6'], ['IF_P', 13], ['SETP', 'GT', 'R5', 'R7'], ['IF_P', 10], ['LOADI', 'R5', '100'], ['ELSE_P', 11], ['LOADI', 'R5', '200'], ['ENDIF'], ['ELSE_P', 14], ['LOADI', 'R5', '300'], ['ENDIF'], ['STORE', 'R5', 'R0']] 


PC 0
['LOADC', 'R0', 'THRD_ID']
P_stack [[True, True, True, True]]
R5  Core0: 0  Core1: 0  Core2: 0  Core3: 0  
R6  Core0: 0  Core1: 0  Core2: 0  Core3: 0  
R7  Core0: 0  Core1: 0  Core2: 0  Core3: 0  

PC 1
['LOAD', 'R5', 'R0']
P_stack [[True, True, True, True]]
R5  Core0: 1  Core1: 2  Core2: 3  Core3: 4  
R6  Core0: 0  Core1: 0  Core2: 0  Core3: 0  
R7  Core0: 0  Core1: 0  Core2: 0  Core3: 0  

PC 2
['LOADI', 'R6', '1']
P_stack [[True, True, True, True]]
R5  Core0: 1  Core1: 2  Core2: 3  Core3: 4  
R6  Core0: 1  Core1: 1  Core2: 1  Core3: 1  
R7  Core0: 0  Core1: 0  Core2: 0  Core3: 0  

PC 3
['LOADI', 'R7', '3']
P_stack [[True, True, True, True]]
R5  Core0: 